In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import poisson
import scipy as sc
import re, random, math, time

df_full = pd.read_csv('full_orders.csv')
df_full = df_full.drop(['Unnamed: 0','Shared_parts'], axis=1)
# WE misunderstood what quantities we wanted --> NOT value counts, rather Order Size*Quantity
df_full['qty_on_order'] = df_full['Order Size']*df_full['Quantity']
df_full['fulfilled_parts'] = np.nan

df_full.head()

,Order Number,Finished Good,Order Size,Item_Number,Quantity,qty_on_order,fulfilled_parts
0,1,1,53,000000000000008820,1,53,NaN
1,1,1,53,SSW0G43707,2,106,NaN
2,1,1,53,SSW0G44348,2,106,NaN
3,1,1,53,SSW0G44670,1,53,NaN
4,1,1,53,SSW0K51133,2,106,NaN


In [2]:
# Set our noise ratio for inventory on hand
noise_ratio = 0.95

# SUM up the qty_on_order grouped by each Item_Number
df_parts = df_full.groupby('Item_Number')[['qty_on_order']].sum()
df_parts.reset_index(level=0, inplace=True)
df_parts['Inventory'] = df_parts['qty_on_order'].apply(lambda x: math.ceil (x*noise_ratio))
df_parts.head()

,Item_Number,qty_on_order,Inventory
0,000000000000008820,97,93
1,000000000000011977,44,42
2,000000000025216222,41,39
3,000000000031028400,80,76
4,000000000031028776,97,93


In [4]:
df_parts['qty_on_order'].describe()

count    145.000000
mean     103.448276
std       68.407981
min       40.000000
25%       44.000000
50%       88.000000
75%      102.000000
max      408.000000
Name: qty_on_order, dtype: float64

In [5]:
# I QUIT!!!!!
df_parts['Inventory'][df_parts['Item_Number']=="SSW0G43707"].values[0]

185

In [7]:
df_full[(df_full['Order Number']==1) & (df_full['Item_Number']=="SSW0G43707")]

,Order Number,Finished Good,Order Size,Item_Number,Quantity,qty_on_order,fulfilled_parts
1,1,1,53,SSW0G43707,2,106,NaN


In [8]:
# HOLY HELL this was frustrating
df_parts.loc[df_parts['Item_Number']=="SSW0G43707", 'Inventory']

106    185
Name: Inventory, dtype: int64

In [9]:
start_time = time.time()

for i in range(1,101):
    current_order = df_full[df_full['Order Number'] == i]
    for j in range(len(current_order)):
        part = current_order.iloc[j]['Item_Number']
        qty = current_order.iloc[j]['qty_on_order']
        on_hand = df_parts['Inventory'][df_parts['Item_Number']==part].values[0]
        # Check to see if we have enough parts on hand
        if ((on_hand - qty) > 0):
            # if so set new column ('Fulfilled') to True
            df_full.loc[(df_full['Order Number']==i) & (df_full['Item_Number']==part),'fulfilled_parts'] = True
            # subtract qty (quantity of current part for the current order) from inventory on hand
            df_parts.loc[df_parts['Item_Number']==part, 'Inventory'] = df_parts['Inventory'][df_parts['Item_Number']==part] - qty
            #print(df_parts['Inventory'][df_parts['Item_Number']==part])

        # If we don't have enough parts set 'Fulfilled' value to False
        else:
            df_full.loc[(df_full['Order Number']==i) & (df_full['Item_Number']==part),'fulfilled_parts'] = False

print("--- %s seconds ---" % (time.time() - start_time))

--- 32.986085653305054 seconds ---


In [24]:
# Save to csv to view ALL values in excel or w/e
df_full.sort_values(by='Order Number').to_csv('fulfilled_simulation.csv')
df_full.head()

,Order Number,Finished Good,Order Size,Item_Number,Quantity,qty_on_order,fulfilled_parts
0,1,1,53,000000000000008820,1,53,True
1,1,1,53,SSW0G43707,2,106,True
2,1,1,53,SSW0G44348,2,106,True
3,1,1,53,SSW0G44670,1,53,True
4,1,1,53,SSW0K51133,2,106,True


In [11]:
# Show remaining inventory --> since I don't remove if qty can't be fulfilled there will be a couple leftovers for each part
df_parts.head()

,Item_Number,qty_on_order,Inventory
0,000000000000008820,97,1
1,000000000000011977,44,1
2,000000000025216222,41,2
3,000000000031028400,80,2
4,000000000031028776,97,1


### Next
- group by 'Order Number'? 'Finished Good'? 'Item_Number'? --> find percent 'Fulfilled'=True

In [21]:
df_full[df_full['Order Number']==84]

,Order Number,Finished Good,Order Size,Item_Number,Quantity,qty_on_order,fulfilled_parts
3038,84,5,1,SSW0K51574,1,1,True
3039,84,5,1,SD50H32698,2,2,True
3040,84,5,1,PROC11021,1,1,True
3041,84,5,1,PROC6222,2,2,True
3042,84,5,1,000000000025216222,1,1,True
3043,84,5,1,SM50F76959,2,2,True
3044,84,5,1,000000000031045784,2,2,True
3045,84,5,1,000000000031506282,1,1,True
3046,84,5,1,000000000031045785,1,1,True
3047,84,5,1,000000000144002367,1,1,True
